# Racing Baseline Script

In [1]:
import numpy as np
import pandas as pd

# Load Data

In [2]:
df = pd.read_csv("../Dataset/hr_race.csv")
df = df.drop(['Unnamed: 0', 'id'], axis=1)
df.head(3)

,rcDate,meet,rcNo,ord,rcTime,age,chulNo,corner_txt,differ,hrName,jkName,owName,plc,prdCtyNm,sex,trName,wgBudam,wgHr,win
0,2000-01-30,서울,1,3.0,01:20:01,4,1,- - - 10 - 9 -,4,웨스턴타이틀,함완식,이근우,1.1,호,암,김문갑,54.0,461.0,1.6
1,2000-01-30,서울,1,11.0,01:22:00,4,2,- - - 7 - 7 -,2,두루미,박수홍,범양해운,31.5,한,암,이왕언,53.0,428.0,610.6
2,2000-01-30,서울,1,12.0,01:22:01,5,3,- - - 12 - 11 -,2½,서천,박동랑,어울림네트웍스,5.8,뉴,암,김문갑,52.0,391.0,52.1


## Feature
- rcDate : 경기가 열린 날 / 2000-01-30 부터 2017-01-06 까지
- meet : 경기가 열린 장소 / 서울, 제주, 부경
- rcNo : 경주시행일의 경주번호 / 1 부터 15 까지
- ord : 출발검사 도착순위
- rcTime : 경주시간(?)
- age : 말의 나이 / 4 부터 27 까지
- chulNo : 경주 출발게이트 번호 / 1 부터 16 까지
- corner_txt : 통과지점별통과순위
- differ : 선순위 도착마와의 도착 마신 차이 / 머리, 목, 주행중지, 출전제외 등
- hrName : 말의이름 / 웨스턴타이틀, 두루미 등
- jkName : 경주마 기승하는 기수명
- owName : 경주마 소유한 마주명
- plc : 해당마의 연승식 적중시 환급 배당률
- prdCtyNm : 경주마 생산국가명 / 호, 한, 뉴, 미 등
- sex : 말의성별 / 암, 거, 수
- trName : 조교사명
- wgBudam : 경주마의 부담중량(KG)
- wgHr : 경주마 해당일 체중(KG)
- win : 해당마의 단승식 적중시 환급 배당률

# Make dataset
- Training set: 80%
- Test set: 20%
- isWin: boolean

In [4]:
from datetime import datetime
from datetime import timedelta
import re

In [5]:
# Object to datetime
df['rcDate'] = pd.to_datetime(df['rcDate'])
df['year'] = pd.to_datetime(df.rcDate).dt.year
df['rcTime'] = pd.to_datetime(df['rcTime']).apply(lambda d: datetime.time(d))

In [6]:
# Float to int
df['ord'] = np.nan_to_num(df['ord']).astype(int)
df['plc'] = np.nan_to_num(df['plc']).astype(int)
df['win'] = np.nan_to_num(df['win']).astype(int)

In [7]:
# Predictors
df['isWin'] = df['ord']==(1 or 2)
df = df.drop(df['ord']==0)

In [8]:
train = df[df['year'] < 2014]
test = df[df['year'] >= 2014]

# Preprocessing
- 우선, 서울, 부경, 제주 중에서 제주마만 예측 (17만개)
- 경기전까지 알 수 있는 신뢰할 수 있는 정보를 사용
- 경주마 정보, 기수 정보, 조교사 정보 등

In [9]:
cols_to_drop = ['chulNo', 'ord', 'rcNo', 'rcTime', 'differ', 'corner_txt', 'year']
train = train.drop(cols_to_drop, axis=1)
train = train[train['meet']=='제주']
train.head()

,rcDate,meet,age,hrName,jkName,owName,plc,prdCtyNm,sex,trName,wgBudam,wgHr,win,isWin
1072,2000-01-30,제주,3,남이장군,이재웅,강상우,32,한,수,김영래,52.0,204.0,21,False
1073,2000-01-30,제주,3,동부로,김기환,한성택,1,한,수,임용찬,55.0,238.0,2,False
1074,2000-01-30,제주,3,쓰리쎄븐,심도연,김덕인,1,한,암,박석천,54.0,216.0,2,False
1075,2000-01-30,제주,3,위성도시,모현진,김용해,6,한,거,김한철,52.0,226.0,12,False
1076,2000-01-30,제주,3,보리고개,이태용,이종철,4,한,암,양일천,54.0,226.0,14,False


## One-Hot Encoding
- prdCtyNm : 경주마 생산국가명 / 호, 한 등 17개국
- sex : 말의성별 / 암, 거, 수
- hrName : 말의이름 / 웨스턴타이틀, 두루미 등
- jkName : 경주마 기승하는 기수명
- owName : 경주마 소유한 마주명
- trName : 조교사명

In [10]:
cols_to_transform = ['sex', 'hrName']
train = pd.get_dummies(train, columns=cols_to_transform)
train.head(3)

,rcDate,meet,age,jkName,owName,plc,prdCtyNm,trName,wgBudam,wgHr,...,hrName_힘찬질주,hrName_힘찬출발,hrName_힘찬포부,hrName_힘찬하루,hrName_힘찬함성,hrName_힘찬햇살,hrName_힘찬행진,hrName_힘찬호령,hrName_힘찬활력,hrName_힘찬희망
1072,2000-01-30,제주,3,이재웅,강상우,32,한,김영래,52.0,204.0,...,0,0,0,0,0,0,0,0,0,0
1073,2000-01-30,제주,3,김기환,한성택,1,한,임용찬,55.0,238.0,...,0,0,0,0,0,0,0,0,0,0
1074,2000-01-30,제주,3,심도연,김덕인,1,한,박석천,54.0,216.0,...,0,0,0,0,0,0,0,0,0,0


# Modeling

In [11]:
predictors = ["plc", "win"]
train_x = train[predictors]
train_y = train["isWin"]
test_x = test[predictors]
test_y = test["isWin"]

## DecisionTree & RandomForest

In [12]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [13]:
# decision = DecisionTreeClassifier().fit(train_x, train_y)
forest = RandomForestClassifier(n_jobs=-1).fit(train_x, train_y)
print(forest)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=-1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)


## Cross Validation

In [14]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

In [15]:
# decision_score = cross_val_score(decision, x, y, cv=5).mean()
forst_score = cross_val_score(forest, test_x, test_y, cv=5).mean()

# print("DecisionTree = {0:.6f}".format(decision_score))
print("RandomForest = {0:.6f}".format(forst_score))

RandomForest = 0.905055


# Result
- plc, win: 0.899656
- age, sex: 0.895907